# COGS 108 - EDA Checkpoint

# Names

- Clara Yi
- Ernest Lin
- Wesley Nguyen
- Ethan Lee
- Stephen Kim

<a id='research_question'></a>
# Research Question

Do the macroscopic socioeconomic features of a state, specifically median income, percentage of population without health insurance, and prevalence of blue collar workers, have a correlation to COVID mortality rate in 2020-2021?

# Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Cleaning

## Dataset 1 (COVID)
- With the imported data, we removed unncessary states. We only want the 50 states not including territories or DC
- We then removed the columns that we didn't need for analysis. We did this by selecting the columns that we needed
- We also wanted the dates to appear in a sortable/searchable way, so we made the dates arranged in yyyy-mm-dd format
- The data was then saved as a csv file.

**Note**: Since the data is arranged by date, we created a function ```read_covid_data``` that will return the 50 states with their respective data for just that specified date

## Dataset 1 Code

In [2]:
# Cleaning State Data
def clean_covid_data():
    # Date Closure
    def apply_date(date: str) -> str:
        split_date = date.split("/")
        return "/".join([split_date[2], split_date[0], split_date[1]])
        
    
    # Read the data (already in tabular form)
    covid_data_url = r"./Raw Data/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv"
    covid_data = pd.read_csv(covid_data_url)
    
    # States we will not be looking at (These aren't part of the 50 states)
    remove_states = ["RMI", "FSM", "GU", "MP", "PW", "NYC", "PR", "AS", "VI", "DC"]
    covid_data = covid_data[~covid_data["state"].isin(remove_states)]
    
    # Remove columns we don't need
    covid_data = covid_data[["submission_date", "state", "tot_cases", "tot_death"]]
    
    # Change Date format to allow for easier sorting
    covid_data["submission_date"] = covid_data["submission_date"].apply(apply_date)
    
    # Sort Date
    covid_data.sort_values("submission_date", inplace=True, ascending=False)
    covid_data.reset_index(inplace=True, drop=True)
    
    # Save Data
    clean_covid_data_url = r".\Cleaned Data\state_covid_data.csv"
    covid_data.to_csv(clean_covid_data_url, index=False)
    
clean_covid_data()

In [3]:
def read_covid_data(month: int, day: int, year: int):
    #retrieve cleaned csv
    covid_data_url = r".\Cleaned Data\state_covid_data.csv"
    covid_data = pd.read_csv(covid_data_url)
    
    #reformat date parameter to match data values in csv, then get all data from specific date
    date_filter = formatDate(month, day, year)
    covid_data = covid_data[covid_data["submission_date"] == date_filter]
    covid_data.sort_values("state", inplace=True)
    covid_data.reset_index(inplace=True, drop=True)
    
    return covid_data

def formatPreZero(num: int) -> str:

    #adds '0' char to any integer less than 10 to match formatting
    if num >= 10:
        return str(num)
    
    return "0" + str(num)
    
#reformat date parameter to match data values in csv    
def formatDate(month: int,  day: int, year: int) -> str:
    return f"{year}/{formatPreZero(month)}/{formatPreZero(day)}"

read_covid_data(3, 15, 2021)

,submission_date,state,tot_cases,tot_death
0,2021/03/15,AK,58212,331
1,2021/03/15,AL,507494,10800
2,2021/03/15,AR,327060,5481
3,2021/03/15,AZ,834006,16553
4,2021/03/15,CA,3528795,55330
5,2021/03/15,CO,452758,6040
6,2021/03/15,CT,293102,7788
7,2021/03/15,DE,91768,1511
8,2021/03/15,FL,1941604,33528
9,2021/03/15,GA,1034763,18262


## Dataset 2 (Labor)
- The raw data file for Dataset 2 is an excel file. The format of the data was not organized in a way that complements dataframes, so we had a lot of unnecessary texts in the excel
read as data entries as well. 
- Our first step was to identify the columns and rows that we want, which are the 50 states. We removed unnecessary states (including U.S. territories) and removed all the extra
non-state entries that were read as rows.
- Another problem is that some names in the State column had some footnote numbers that were unintentionally read from the excel sheet. We solved this by removing all occurences of numbers and parentheses from the State column.
- Since other datasets use state codes and the original data uses state names, we had to transform state names in the States column to their corresponding state codes. We did this by defining a function ```to_state_code``` that uses a dictionary to map each state name to their state code.
- We had to reorganize the structure of the dataframe, as the original file had the data stacked on top of each other so each state had 3 rows in the Dataframe. We did so by separating the 
raw dataframe into three different dataframes, and then combining them into a single dataframe so we only have 50 rows.
- We then removed unnessary columns, such as data from other time periods (Our focus was December of 2020). We also combined the columns of job sectors into two groups relevant to our analysis: Blue collar (construction, mining, trade, leisure) and White collar (Financial, professional, education, government) jobs.
- Our final step for Dataset 3 is to export the cleaned dataset as a csv and save it to the "Cleaned Data" folder.

## Dataset 2 Code

In [4]:
#Copied from https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

def to_state_code(state_name):
    return us_state_to_abbrev[state_name]

In [5]:
def clean_labor_data():
    #Read excel file, renames first column to States and take out null rows
    raw_labor_data = pd.read_excel("./Raw Data/labor_dataset_raw.xlsx", header = 4)
    raw_labor_data.rename(columns={"Unnamed: 0": "State"}, inplace=True)
    raw_labor_data = raw_labor_data.dropna()

    #Take out data from 2021 and only keep 2020
    raw_labor_data = raw_labor_data[["State", "Dec.\n2020", "Dec.\n2020.1", "Dec.\n2020.2"]]
    
    non_states = ["Virgin Islands", "District of Columbia", "Puerto Rico"]

    #Removes all non official states from dataset
    for region in non_states:
        raw_labor_data = raw_labor_data[raw_labor_data["State"].str.contains(region)==False]

    #Reset index to start at 0
    raw_labor_data = raw_labor_data.reset_index(drop = True)

    #eliminated extra characters in state names
    raw_labor_data["State"] = raw_labor_data["State"].str.replace('\d+', '')
    raw_labor_data["State"] = raw_labor_data["State"].str.replace('(', '')
    raw_labor_data["State"] = raw_labor_data["State"].str.replace(')', '')

    #Convert state names into codes (First two letters of each state name)
    raw_labor_data["State"] = raw_labor_data["State"].apply(lambda state_name: us_state_to_abbrev[state_name])

    #Original raw data has different columns stacked on top of each row, so we need to reorder the dataset.
    #Block 1 contains total, constructing and mining data
    block1 = raw_labor_data[:50]
    block1.columns = ["State", "Total", "Constructing", "Mining"]

    #Block 2 contains Trade, Financial and Professional
    block2 = raw_labor_data[50:100]
    block2.columns = ["State", "Trade", "Financial", "Professional"]

    #Block 3 contains Education, Leisure and Government
    block3 = raw_labor_data[100:]
    block3.columns = ["State", "Education", "Leisure", "Gov"]

    #merge all blocks into one dataframe
    labor_data = block1.merge(block2, on="State")
    labor_data = labor_data.merge(block3, on="State")

    #We only need data on white collar and blue collar, so we can combine each job sector to their respective group.
    labor_data["Blue_col"] = labor_data["Constructing"] + labor_data["Mining"] + labor_data["Trade"] + labor_data["Leisure"]
    labor_data["White_col"] = labor_data["Financial"] + labor_data["Professional"] + labor_data["Education"] + labor_data["Gov"]

    #Get rid of all other columns except State, White_col, Blue_col and Total
    labor_data.drop(columns = ["Constructing", "Mining", "Trade", "Financial", "Professional", "Education", "Leisure", "Gov"], inplace=True)
    #export as csv
    labor_data.to_csv('./Cleaned Data/state_labor_data.csv')

clean_labor_data()

C:\Users\ethan\AppData\Local\Temp/ipykernel_23036/3203687775.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  raw_labor_data["State"] = raw_labor_data["State"].str.replace('\d+', '')
C:\Users\ethan\AppData\Local\Temp/ipykernel_23036/3203687775.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  raw_labor_data["State"] = raw_labor_data["State"].str.replace('(', '')
C:\Users\ethan\AppData\Local\Temp/ipykernel_23036/3203687775.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  raw_labor_data["State"] = raw_labor_data["State"].str.replace(')', '')



## Dataset 3 (Income/Insurance)
- For Dataset 3, we have two primary steps in cleaning the data. The first step was manually inputting the data from the data sources to a CSV file via Google Sheets. This manual step was necessary due to the fact that the original data source did not have an option to directly extract/download the raw data. Since there were only 50 observations, we decided manual input was the best option. 

- Our second step for Dataset 3 was to import the data into this notebook. We uploaded the CSV file into our "Raw Data" folder, and then used read_csv to bring it into a dataframe, which is a usable format for our future analysis. After making sure there were no issues, we then saved it to the "Cleaned Data" folder.

## Dataset 3 Code

In [6]:
# Cleaning socioeconomic data

socioeconomic_data_url = r'./Raw Data/socioeconomic_data.csv'
socioeconomic_data = pd.read_csv(socioeconomic_data_url)

# Saving to CSV
clean_socioeconomic_data_url = r"./Cleaned Data/clean_socioeconomic_data.csv"
socioeconomic_data.to_csv(clean_socioeconomic_data_url, index=False)

# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [7]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION